In [3]:
!pip install instructor anthropic[bedrock]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.0 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.2 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 762.4 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.4.1
    Uninstalling tenacity-8.4.1:
      Successfully uninstalled tenacity-8.4.1
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: jiter
    Found existing installation: jiter 0.8.0
    Uninstalling jiter-0.8.0:
      Successfully uninstalled jiter-0.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s

In [70]:
import instructor
from anthropic import AnthropicBedrock
from pydantic import BaseModel
from typing import List
from IPython.display import Markdown, display
from tabulate import tabulate
import textwrap
import json
from utils import read_file, save_file

In [53]:
i_client = instructor.from_anthropic(AnthropicBedrock())
a_client = AnthropicBedrock()

In [44]:
class User(BaseModel):
    name: str
    age: int

In [45]:
# note that client.chat.completions.create will also work
resp = i_client.messages.create(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Extract Jason is 25 years old.",
        }
    ],
    response_model=User,
)

In [46]:
assert isinstance(resp, User)
assert resp.name == "Jason"
assert resp.age == 25
resp

User(name='Jason', age=25)

In [47]:
ab_paper = read_file('data/whitepaper/AB_2013-07_Model_Risk_Management_Guidance.md')
moody_paper = read_file('data/whitepaper/riskcalc-3.1-whitepaper.md')

In [48]:
class Task(BaseModel):
    description: str
    instructons: str
    exmaples: str

In [91]:
def generate_tasks(question, document, model="anthropic.claude-3-haiku-20240307-v1:0"):
    message = f"""
    Generate a comprehensive list of model analysis tasks based on provide model whitepaper. Each task includes short description, detailed instructions and list of examples to answer this question: {question}.
    Be as detailed as possible. Number of identified tasks should ensure comprehensive analysis.
    Your response should be a valid formed json object and nothing else.
    Example:
    task 
        descripiton: task desciption
        insturctions: task instrucitons
        examples: task examples
    ...        
    """
    whitepaper = f"""
    <whitepaper>
    {document}
    </whitepaper>
    """

    return a_client.messages.create(
        model=model,
        system= whitepaper,
        max_tokens=3000,
        messages=[
            {
                "role": "user",
                "content": message,
            }
        ]#,
        #response_model=Task,
        )

In [92]:
qq = ['Identify any specific limitations and model usage risk in stagflation environment']#,
      #'Indentify any specific limitations and model usage risks in hyper-inflation scenario']

for i, q in enumerate(qq):
    tasks = generate_tasks(q, moody_paper, model= 'anthropic.claude-3-sonnet-20240229-v1:0')
    #display(Markdown(f"```json\n{tasks.content[0].text}```"))
    display(Markdown(f"```json\n{tasks.content[0].text}\n```"))


```json
{
    "task_1": {
        "description": "Assess the model's sensitivity to macroeconomic variables",
        "instructions": "Analyze how the model's performance is affected by changes in macroeconomic variables such as interest rates, GDP, unemployment rate, and inflation rate. Pay particular attention to the model's behavior during periods of stagflation, where there is slow economic growth, high unemployment, and high inflation.",
        "examples": [
            "Evaluate the model's accuracy in predicting defaults during historical periods of stagflation.",
            "Conduct stress tests by simulating scenarios with varying levels of interest rates, GDP growth, unemployment, and inflation to understand the model's behavior.",
            "Investigate the impact of stagflation on the model's distance-to-default measure and its ability to capture market risk."
        ]
    },
    "task_2": {
        "description": "Examine the model's reliance on financial statement data",
        "instructions": "Assess the model's dependence on financial statement data and its ability to capture changes in economic conditions that may not be immediately reflected in financial statements. During stagflation, financial statements may lag in reflecting the deteriorating economic conditions.",
        "examples": [
            "Evaluate the model's performance when using only the Financial Statement Only (FSO) mode during periods of stagflation.",
            "Compare the FSO mode's predictions with the complete model's predictions that incorporate market-based information during stagflation periods.",
            "Investigate the potential impact of delayed reporting or inconsistent accounting practices on the model's accuracy during economic downturns."
        ]
    },
    "task_3": {
        "description": "Analyze the model's industry-specific adjustments",
        "instructions": "Investigate how the model's industry-specific adjustments perform during stagflation periods. Different industries may be affected differently by stagflation, and the model's ability to capture industry-specific dynamics should be evaluated.",
        "examples": [
            "Assess the model's accuracy in predicting defaults for industries that are particularly vulnerable to stagflation, such as consumer discretionary or cyclical industries.",
            "Evaluate the model's ability to differentiate between industries that are more resilient to stagflation and those that are more susceptible.",
            "Analyze the impact of industry-specific factors, such as inventory levels or pricing power, on the model's performance during stagflation."
        ]
    },
    "task_4": {
        "description": "Evaluate the model's stress testing capabilities",
        "instructions": "Examine the model's stress testing capabilities and its ability to accurately predict default probabilities under different economic scenarios, including stagflation. The model's stress testing features should be thoroughly assessed to identify potential limitations or risks.",
        "examples": [
            "Conduct stress tests by simulating stagflation scenarios with varying levels of inflation, unemployment, and economic growth to evaluate the model's performance.",
            "Analyze the model's ability to capture the impact of stagflation on individual firms and assess the reliability of the stress testing results.",
            "Investigate the potential for the model to underestimate or overestimate default probabilities during periods of stagflation."
        ]
    },
    "task_5": {
        "description": "Evaluate the model's calibration and validation processes",
        "instructions": "Assess the model's calibration and validation processes to ensure that they are robust and account for potential economic downturns, including stagflation periods. The model's ability to maintain accurate default probability estimates during such periods should be evaluated.",
        "examples": [
            "Review the validation data and processes used to calibrate the model, ensuring that they include periods of stagflation or similar economic conditions.",
            "Evaluate the model's performance during out-of-sample and out-of-time validation tests that cover stagflation periods.",
            "Investigate the potential impact of stagflation on the model's calibration and the need for recalibration or adjustments to maintain accuracy."
        ]
    },
    "task_6": {
        "description": "Analyze the model's term structure and default rate projections",
        "instructions": "Evaluate the model's ability to accurately project default rates over different time horizons during periods of stagflation. The model's term structure and assumptions about mean reversion in credit quality should be scrutinized.",
        "examples": [
            "Assess the model's accuracy in projecting default rates over different time horizons (e.g., 1-year, 3-year, 5-year) during historical stagflation periods.",
            "Investigate the model's assumptions about mean reversion in credit quality and their validity during prolonged periods of economic stagnation.",
            "Evaluate the model's ability to capture potential changes in default rate dynamics during stagflation, which may differ from normal economic cycles."
        ]
    },
    "task_7": {
        "description": "Evaluate the model's treatment of industry-specific risk factors",
        "instructions": "Investigate how the model handles industry-specific risk factors and their potential impact during stagflation periods. Certain industries may face unique challenges or exhibit different risk dynamics during such economic conditions.",
        "examples": [
            "Analyze the model's treatment of industry-specific risk factors, such as inventory levels, pricing power, or consumer demand, and their relevance during stagflation.",
            "Evaluate the model's ability to capture changes in industry-specific risk factors and their impact on default probabilities during periods of stagflation.",
            "Investigate the potential need for adjustments or additional industry-specific factors to improve the model's performance during stagflation."
        ]
    },
    "task_8": {
        "description": "Assess the model's treatment of idiosyncratic and systematic risk",
        "instructions": "Evaluate the model's ability to capture both idiosyncratic (firm-specific) and systematic (market-based) risk factors during periods of stagflation. The relative importance of these factors may shift during such economic conditions.",
        "examples": [
            "Analyze the model's weighting and treatment of idiosyncratic and systematic risk factors during historical stagflation periods.",
            "Investigate the potential need for adjustments to the model's weighting or incorporation of these risk factors to improve performance during stagflation.",
            "Evaluate the model's ability to capture potential changes in the relative importance of idiosyncratic and systematic risk factors during prolonged economic stagnation."
        ]
    }
}
```